In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
from StringIO import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations}
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from pandas import Series
from pandas import DataFrame
import timeit
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import timeit
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [3]:
job_posting_table = pd.read_csv("C:/R_data/Job_for_Myklovr/job_posting.csv",sep=",")
jab_cate_gourp = job_posting_table.groupby(["Type"])
jab_cate_gourp = jab_cate_gourp.count()
jab_cate_gourp = jab_cate_gourp.drop(jab_cate_gourp.columns[1:],axis = 1)
jab_cate_gourp = jab_cate_gourp.rename(columns = {"Unnamed: 0":"Count"})

In [4]:
job_posting_table

,Unnamed: 0,Title,Company,Location,Type,Description
0,1,Civil Site Design Engineer - Land Development ...,Absolute Opportunities,"Orange, CA",design,Civil Site Design Engineer - Land Development ...
1,10,Digital Media Planner,Green Key Resources,"New York, NY",digital-media,Our major fashion retailer is seeking a Digita...
2,12,Process Development Chemist- 6882,Simply Biotech,"Torrance, CA",chemist,Process Development Chemist- Simply Biotech ...
3,20,Real Estate Loan Processor (for credit union),Amtec Inc.,"La Habra, CA",real estate,Job Title: Real Estate Loan Processor Durat...
4,23,Sherman Oaks Law Firm Seeking Collections / De...,Gershfeld Law Group,"Sherman Oaks, CA",law,Are you looking for a good stable company to w...
5,24,Sr. Data Analyst - Competitive Pay!,"Premier Staffing Source, Inc.","New York, NY",data analyst,"Premier Staffing Source, Inc. is seeking a S..."
6,32,Teacher Assistant,Today's Learning Center,"Voorhees Township, NJ",teacher,Teacher Assistants Needed in two different cla...
7,33,Vice President of Human Resources,Professional Recruiters,"Phoenix, AZ",human resource,"VP of Human Resources Location: Phoenix, AZ ..."
8,34,Automation Software Engineer,21st Century Group,"Baltimore, MD",software engineer,Automation Software Engineer Immediate posit...
9,35,Court Research/Data Entry - Part Time - TX-Wil...,IT-Boss Research,"Floresville, TX",data entry,Data Entry - Court Research Abstractor This ...


In [4]:
jab_cate_gourp

,Count
Type,
chemist,43
data analyst,190
data entry,231
design,227
digital-media,57
health care,227
human resource,230
law,243
real estate,216


In [5]:
job_posting_table = job_posting_table.drop(job_posting_table.columns[0],axis=1)
job_posting_table["ID"] = job_posting_table.index


In [6]:
job_type_description_table = pd.DataFrame(columns={"Type","Job_Description_sum"})

In [7]:
#Job_type_description = ""
n = 0
for a in jab_cate_gourp.index:
    
     
    job_type_description_dic = {}
    job_type = a
    Job_type_description = ""
    data_table_of_job_type = job_posting_table[job_posting_table.Type == job_type]
    job_description_list = list(data_table_of_job_type.Description.values)
    for b in range(len(job_description_list)):
        Job_type_description = Job_type_description + job_description_list[b]
    job_type_description_dic = {"Type":job_type,"Job_Description_sum":Job_type_description}
    job_type_description_series = Series(job_type_description_dic)
    job_type_description_small_table = DataFrame(job_type_description_series)
    job_type_description_small_table = job_type_description_small_table.transpose()
    job_type_description_table = job_type_description_table.append(job_type_description_small_table)

In [8]:
job_type_description_table = job_type_description_table.reset_index()
job_type_description_table = job_type_description_table.drop(job_type_description_table.columns[0],axis=1)

In [9]:
job_type_description_table

,Job_Description_sum,Type
0,Process Development Chemist- Simply Biotech ...,chemist
1,"Premier Staffing Source, Inc. is seeking a S...",data analyst
2,Data Entry - Court Research Abstractor This ...,data entry
3,Civil Site Design Engineer - Land Development ...,design
4,Our major fashion retailer is seeking a Digita...,digital-media
5,"GeBBS Healthcare Solutions, an industry lea...",health care
6,"VP of Human Resources Location: Phoenix, AZ ...",human resource
7,Are you looking for a good stable company to w...,law
8,Job Title: Real Estate Loan Processor Durat...,real estate
9,Sales Person and Internet : We are growing!!...,sales


In [10]:
student_resume_table = pd.DataFrame(columns=["student_name","resume_content"])
path = 'D:/R_data/Job_for_Myklovr/resume'
for filename in os.listdir(path):
    student = open(path+"/"+filename,"rU")
    resume_student = student.read()
    resume_student = resume_student.decode("utf-8","ignore")
    resume_student = resume_student.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    resume_student = json.dumps(resume_student)
    resume_student_content = resume_student
    student_name = filename.split(".")[0]
    #print student_name,resume_student_content
    
    student_info_dic = {"student_name":student_name,"resume_content":resume_student_content}
    series = pd.Series(student_info_dic)
    student_table = pd.DataFrame(series)
    student_table = student_table.transpose()
    student_resume_table = student_resume_table.append(student_table)
    student_resume_table = student_resume_table.reset_index()
    student_resume_table = student_resume_table.drop(student_resume_table.columns[0],axis=1)

In [11]:
student_resume_table

,resume_content,student_name
0,"""ADITI RAMESH BHUTEKAR 48, Romaine Avenue, Jer...",aditi ramesh bhutekar
1,"""ANH PHUNG New York, NY | +1 (218) 409-9739 |...",Anh_Phung
2,"""Fanny Heneine Flat 1806, 100 Landsdowne Stree...",fanny
3,"""Jialyu Zhang (848) 391-5026 Highland Park, NJ...",jialyu
4,"""Haoxi Huang 560 MAIN ST, New York, NY 10044 3...",resume_HaoxiHuang


## find the suitbale career path for every student

find a way to moderate the long running time 

In [12]:
career_description_list = list(job_type_description_table.Job_Description_sum)
start = timeit.default_timer()

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in career_description_list:
    allwords_stemmed = tokenize_and_stem(i) 
    totalvocab_stemmed.extend(allwords_stemmed) 
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
stop = timeit.default_timer()

print stop - start

27.4271617863


In [13]:
tfidf_vectorizer = TfidfVectorizer( max_features=5000,
                                  stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(career_description_list) 


Wall time: 23.4 s


formulate the career path/ token tf-idf matrix

In [14]:
dictionary_df = DataFrame(Series(tfidf_vectorizer.get_feature_names()))

In [15]:
for a in range(len(career_description_list)):
    row_array = tfidf_matrix[a].toarray()
    row_list = row_array.tolist()[0]
    career_path_name = job_type_description_table.ix[a,["Type"]].values[0]
    dictionary_df[career_path_name] = row_list

In [16]:
dictionary_df = dictionary_df.rename(columns={0:"tokens"})

In [17]:
dictionary_df

,tokens,chemist,data analyst,data entry,design,digital-media,health care,human resource,law,real estate,sales,software engineer,teacher
0,-experi,0.003017,0.000524,0.000000,0.004730,0.000000,0.000688,0.000000,0.000000,0.003095,0.000000,0.001778,0.000000
1,.net,0.000000,0.004429,0.000000,0.002856,0.000000,0.003322,0.000000,0.000000,0.000000,0.000000,0.058705,0.000000
2,.net c,0.000000,0.000798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027991,0.000000
3,.net c wpf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028387,0.000000
4,1st,0.002353,0.000000,0.003445,0.001383,0.001217,0.000000,0.000304,0.002207,0.001448,0.001592,0.000000,0.000444
5,3d,0.000000,0.000000,0.000000,0.040572,0.000000,0.000000,0.000000,0.000000,0.004164,0.000000,0.004786,0.000000
6,3rd,0.000000,0.001747,0.001179,0.001183,0.005205,0.001376,0.000260,0.000630,0.000826,0.000272,0.002372,0.000380
7,3rd parti,0.000000,0.002042,0.000689,0.001383,0.006084,0.001608,0.000304,0.000736,0.000483,0.000000,0.002773,0.000000
8,401k,0.001864,0.002265,0.007095,0.005477,0.006747,0.012739,0.005786,0.009909,0.003441,0.007568,0.002929,0.002460
9,a.m.,0.000000,0.000000,0.004865,0.000542,0.001432,0.001262,0.001432,0.004329,0.000000,0.001124,0.000000,0.000000


choose top 2500 keywords as the content of every career path 

In [18]:
job_type_description_table["keyword_description"] = "void"

In [19]:
for a in dictionary_df.columns[1:]:
    keyword_token_list = list(dictionary_df.sort_values(by=[a],ascending = False).tokens)[:2500]
    keyword_descripton = " ".join(keyword_token_list)
    index_num = list(job_type_description_table.Type).index(a)
    job_type_description_table.ix[index_num,["keyword_description"]] = keyword_descripton

In [20]:
job_type_description_table

,Job_Description_sum,Type,keyword_description
0,Process Development Chemist- Simply Biotech ...,chemist,chemist product middot experi chemistri work r...
1,"Premier Staffing Source, Inc. is seeking a S...",data analyst,data experi middot work busi requir manag bull...
2,Data Entry - Court Research Abstractor This ...,data entry,data data entri entri work bull skill requir m...
3,Civil Site Design Engineer - Land Development ...,design,design work experi middot requir engin bull pr...
4,Our major fashion retailer is seeking a Digita...,digital-media,media digit digit media client work experi man...
5,"GeBBS Healthcare Solutions, an industry lea...",health care,care nurs healthcar middot experi patient work...
6,"VP of Human Resources Location: Phoenix, AZ ...",human resource,manag human human resourc resourc employe hr m...
7,Are you looking for a good stable company to w...,law,law law firm experi firm work client attorney ...
8,Job Title: Real Estate Loan Processor Durat...,real estate,estat real estat real manag team work experi b...
9,Sales Person and Internet : We are growing!!...,sales,sale manag custom market busi servic work expe...


In [21]:
student_resume_table["top_3_career_path_recommendation"] = "void"

In [22]:
start = timeit.default_timer()

for a in student_resume_table.index:
    career_path_student_list = []
    resume_content = student_resume_table.ix[a,["resume_content"]].values[0]
    career_path_student_list.append(resume_content)
    for b in job_type_description_table.index:
        keyword_description = job_type_description_table.ix[b,["keyword_description"]].values[0]
        career_path_student_list.append(keyword_description)

    tfidf_vectorizer = TfidfVectorizer( max_features=200000,
                                  stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))


    %time tfidf_matrix = tfidf_vectorizer.fit_transform(career_path_student_list) 
    
    dist_1  =  cosine_similarity(tfidf_matrix)
    sim_matrix_table = pd.DataFrame(dist_1)
    student_list = sim_matrix_table[sim_matrix_table.index[0]]
    student_list_table = DataFrame(student_list)
    student_list_table["career_path"] = "void"
    student_list_table = student_list_table.drop(student_list_table.index[0],axis = 0)
    for b in student_list_table.index:
        track_index = b-1
        career_path = job_type_description_table.ix[track_index,["Type"]].values[0]
        student_list_table.ix[b,["career_path"]] = career_path
        student_list_table = student_list_table.rename(columns={0:"relevence_score"})
        student_list_table = student_list_table.sort_values(by=["relevence_score"],ascending = False)
        career_path_list_top_3 = list(student_list_table.career_path)[:3]
        top_3_career_path = ",".join(career_path_list_top_3)
    student_resume_table.ix[a,["top_3_career_path_recommendation"]] = top_3_career_path

Wall time: 1.51 s
Wall time: 1.77 s
Wall time: 1.49 s
Wall time: 1.79 s
Wall time: 1.67 s


In [24]:
student_resume_table

,resume_content,student_name,top_3_career_path_recommendation
0,"""ADITI RAMESH BHUTEKAR 48, Romaine Avenue, Jer...",aditi ramesh bhutekar,"data analyst,software engineer,digital-media"
1,"""ANH PHUNG New York, NY | +1 (218) 409-9739 |...",Anh_Phung,"data analyst,software engineer,law"
2,"""Fanny Heneine Flat 1806, 100 Landsdowne Stree...",fanny,"chemist,law,design"
3,"""Jialyu Zhang (848) 391-5026 Highland Park, NJ...",jialyu,"data analyst,software engineer,data entry"
4,"""Haoxi Huang 560 MAIN ST, New York, NY 10044 3...",resume_HaoxiHuang,"data analyst,software engineer,data entry"


## recommend every student detailed job title from top three career path

In [31]:
for a in student_resume_table.index:
    top_3_career_path_list = student_resume_table.ix[a,["top_3_career_path_recommendation"]].values[0].split(",")
    student_resume = student_resume_table.ix[a,["resume_content"]].values[0]
    student_name = student_resume_table.ix[a,["student_name"]].values[0]
    
    new_job_posting_table = pd.DataFrame(columns =job_posting_table.columns )
    for detail_career_path in top_3_career_path_list:
        sub_job_posting_table = job_posting_table[job_posting_table.Type == detail_career_path]
        new_job_posting_table = new_job_posting_table.append(sub_job_posting_table)
    

    new_job_posting_table = new_job_posting_table.reset_index()
    new_job_posting_table = new_job_posting_table.drop(new_job_posting_table.columns[0],axis = 1)
   
    student_job_title_list = []
    student_job_title_list.append(student_resume)
    for job_description in new_job_posting_table.Description:
        student_job_title_list.append(job_description)
        
    
    tfidf_vectorizer = TfidfVectorizer( max_features=10000,
                                  stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))
    %time tfidf_matrix = tfidf_vectorizer.fit_transform(student_job_title_list) 
    
    dist_1  =  cosine_similarity(tfidf_matrix)
    sim_matrix_table = pd.DataFrame(dist_1)
    job_title_recommendation_row = pd.DataFrame(sim_matrix_table.ix[0])
    job_title_recommendation_row = job_title_recommendation_row.drop(job_title_recommendation_row.index[0],axis=0)
    job_title_recommendation_row = job_title_recommendation_row.rename(columns={0:"relevence_score"})
    
    job_title_recommendation_row["Title"] = "void"
    job_title_recommendation_row["Company"] = "void"
    job_title_recommendation_row["Location"] = "void"
    job_title_recommendation_row["Description"] = "void"
    job_title_recommendation_row["ID"] = "void"
    job_title_recommendation_row["Type"] = "void"
    for a in job_title_recommendation_row.index:
        track_id = a-1
        job_title = new_job_posting_table.ix[track_id,["Title"]].values[0]
        #print job_title
        company = new_job_posting_table.ix[track_id,["Company"]].values[0]
        location = new_job_posting_table.ix[track_id,["Location"]].values[0]
        description = new_job_posting_table.ix[track_id,["Description"]].values[0]
        ID = new_job_posting_table.ix[track_id,["ID"]].values[0]
        Type = new_job_posting_table.ix[track_id,["Type"]].values[0]
        job_title_recommendation_row.ix[a,["Title"]] = job_title
        job_title_recommendation_row.ix[a,["Company"]] = company
        job_title_recommendation_row.ix[a,["Location"]] = location
        job_title_recommendation_row.ix[a,["Description"]] = description
        job_title_recommendation_row.ix[a,["ID"]] = ID
        job_title_recommendation_row.ix[a,["Type"]] = Type
        job_title_recommendation_row_final = job_title_recommendation_row.sort_values(by=["relevence_score"],ascending = False)
        job_title_recommendation_row_final.columns.name = student_name
        job_title_recommendation_row_final.to_csv("D:/R_data/Job_for_Myklovr/job_title_recommendation/"+student_name+".csv",sep=",")

Wall time: 5.66 s
Wall time: 5.89 s
Wall time: 4.29 s
Wall time: 6.16 s
Wall time: 5.84 s


In [30]:
job_title_recommendation_row_final

resume_HaoxiHuang,relevence_score,Title,Company,Location,Description,ID,Type
127,0.186222,Data Analyst-Scientist Medical Product or Defe...,"Extremely Stable Firm, Recession Resistant, Ni...","Milpitas, CA",Newly Created Role with an Industry Leading S...,1493,data analyst
58,0.141343,Data Analyst - Software,Technology Consulting,"Reston, VA",If you enjoy the search for interesting patte...,697,data analyst
10,0.133426,Data Analyst,Recruiting Edge,"Boston, MA",Company Description We work for a local tech...,116,data analyst
157,0.131744,Director Data Science &amp; Analytics,Honeywell Aerospace,"Phoenix, AZ","At Honeywell, our people are driven by a desir...",1886,data analyst
5,0.131181,"Data Analytics/Analyst - 600 - Seattle, WA",PlacingIT,"Seattle, WA",Data Analytics/Analyst - 600 DETAILS Loc...,52,data analyst
6,0.131181,"Data Analytics/Analyst - 598 - Boston, MA",PlacingIT,"Boston, MA",Data Analytics/Analyst - 598 DETAILS Loc...,59,data analyst
15,0.125272,Data Analyst,Ascendum Solutions,"Georgetown, KY",Data Analyst Responsible for the structuring...,159,data analyst
145,0.125222,"Data Scientist and Analytics Developer, R (Isl...","BryceNet, Inc.","Island Park, NY","Data Scientist and Analytics Developer, R (I...",1715,data analyst
171,0.124526,Data Scientist &amp; Analytics Developer,bp staffing,"Long Beach, NY",Data Scientist &amp; Analytics Developer--Insu...,2046,data analyst
138,0.120137,Senior Data Scientist/Analyst (Non Medical ind...,"TER Consulting,LLC","Boston, MA",This is a fast growing firm with locations in ...,1650,data analyst
